In [1]:
import PyPDF2
import requests
import numpy as np, seaborn as sns, pandas as pd
import nltk, nltk.sentiment, sklearn
import collections
%matplotlib inline

In [2]:
#convert list to pd.series
#doc1_str = pd.Series([''.join(doc1)])

#Preprocessing - tokenize and lower case pdf-strings
tokenizer = nltk.tokenize.TweetTokenizer()
def preprocessing(string):
    string = string.lower()
    string = tokenizer.tokenize(string)
    return string

#Apply positive/negative sets on pdf-string
#tok_doc1 = doc1_str.apply(preprocessing)

def count_dictionary(document_elem, dictionary):
    lst = []
    for word in document_elem:
        if (word in dictionary):
            lst.append(word)
    return len(lst)


In [3]:
#PDF extractor script. Creates a list separated per page.
def pdf_extractor(path):
    pdf_file = open(path, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    c = collections.Counter(range(number_of_pages))
    res1 = []
    for i in range(len(c)):
        page = read_pdf.getPage(i)
        page_content = page.extractText()
        res1.append(page_content)
    return res1
   

In [4]:
import requests

#Create set for negative and positive lexicon
negative = set(requests.get('http://ptrckprry.com/course/ssd/data/negative-words.txt').text.split(';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;')[2].split('\n')[2:])
positive = set(requests.get('http://ptrckprry.com/course/ssd/data/positive-words.txt').text.split(';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;')[2].split('\n')[2:])

In [29]:
import os

#Sebastian file path
basedir = r'C:\Users\sebag\Documents\GitHub\SDS Group 35\Exam_Project\Minutes_PDF'

def sentiment_analysis(pos, neg, file_path):
    pos_words = []
    neg_words = []
    average_tone = []
    i = 0
    error = []
    
    for fn in os.listdir(file_path):
        try:
            print('Now treating file: ' + fn + '\nIterate: ' + str(i))
            doc = pdf_extractor(file_path + '\\' + fn)
            doc = pd.Series([''.join(doc)])
            doc = doc.apply(preprocessing)

            pos_words.append(int(count_dictionary(doc[0], pos)))
            neg_words.append(int(count_dictionary(doc[0], neg)))
            average_tone.append((pos_words[i]-neg_words[i])/len(doc[0]))
            error.append(False)

            i += 1

            #date = re.findall(r'\d+', fn)
        except KeyError:
            pos_words.append(np.NaN)
            neg_words.append(np.NaN)
            average_tone.append(np.NaN)
            error.append(True)
            i += 1
        
    df = pd.DataFrame(zip(pos_words, neg_words, average_tone, error))
    return df

sent_df = sentiment_analysis(positive, negative, basedir)


Now treating file: ppp000203e.pdf
Iterate: 0
Now treating file: ppp000322e.pdf
Iterate: 1
Now treating file: ppp000504e.pdf
Iterate: 2
Now treating file: ppp000607e.pdf
Iterate: 3
Now treating file: ppp000706e.pdf
Iterate: 4
Now treating file: ppp000816e.pdf
Iterate: 5
Now treating file: ppp001024e.pdf
Iterate: 6
Now treating file: ppp001220e.pdf
Iterate: 7
Now treating file: ppp010201e.pdf
Iterate: 8
Now treating file: ppp010326e.pdf
Iterate: 9
Now treating file: ppp010426e.pdf
Iterate: 10
Now treating file: ppp010530e.pdf
Iterate: 11
Now treating file: ppp010614e.pdf
Iterate: 12
Now treating file: ppp010705e.pdf
Iterate: 13
Now treating file: ppp010823e.pdf
Iterate: 14
Now treating file: ppp010917e.pdf
Iterate: 15
Now treating file: ppp011015e.pdf
Iterate: 16
Now treating file: ppp011108e.pdf
Iterate: 17
Now treating file: ppp011204e.pdf
Iterate: 18
Now treating file: ppp020207e.pdf
Iterate: 19
Now treating file: ppp020318e.pdf
Iterate: 20
Now treating file: ppp020425e.pdf
Iterate: 2

In [30]:
sent_df.columns = ['Number of positive words', 'Number of negative words', 'Average tone ratio', 'Error in processing PDF']
print(sent_df.count())
print(sent_df.tail(30))

Number of positive words    113
Number of negative words    113
Average tone ratio          113
Error in processing PDF     137
dtype: int64
     Number of positive words  Number of negative words  Average tone ratio  \
107                     160.0                     203.0           -0.004423   
108                       NaN                       NaN                 NaN   
109                       NaN                       NaN                 NaN   
110                       NaN                       NaN                 NaN   
111                       NaN                       NaN                 NaN   
112                     238.0                     203.0            0.002855   
113                       NaN                       NaN                 NaN   
114                       1.0                       0.0            0.003300   
115                       1.0                       1.0            0.000000   
116                     294.0                     277.0            0.

In [32]:
testdir = r'C:\Users\sebag\Documents\GitHub\Exam\Test folder'
sentiment_analysis(positive, negative, testdir)

Now treating file: ppp000001e.pdf
Iterate: 0


Now treating file: ppp000002e.pdf
Iterate: 1
Now treating file: ppp000003e.pdf
Iterate: 2
Now treating file: ppp181219e.pdf
Iterate: 3


,0,1,2,3
0,122.0,85.0,0.006086,False
1,70.0,41.0,0.006694,False
2,70.0,48.0,0.004838,False
3,NaN,NaN,NaN,True


In [29]:
pos_words = []
doc = pdf_extractor(r'C:\Users\sebag\Documents\GitHub\SDS Group 35\Exam_Project\Minutes_PDF\ppp000203e.pdf')
doc = pd.Series([''.join(doc)])
doc = doc.apply(preprocessing)
count_dictionary(doc[0], negative)
pos_words.append(count_dictionary(doc[0], positive))
print(pos_words)
print(len(doc[0]))

[122]
6080


In [38]:
testdir = r'C:\Users\sebag\Documents\GitHub\Exam\Test folder'
pos_words = []
neg_words = []
average_tone = []
i = 0

for fn in os.listdir(testdir):
    doc = pdf_extractor(testdir + '\\' + fn)
    doc = pd.Series([''.join(doc)])
    doc = doc.apply(preprocessing)
    pos_words.append(count_dictionary(doc[0], positive))
    neg_words.append(count_dictionary(doc[0], negative))
    average_tone = (pos_words[i]-neg_words[i])/len(doc[0])
        
    i += 1
    #date = re.findall(r'\d+', fn)
df = pd.DataFrame(zip(pos_words, neg_words, average_tone))
df

TypeError: zip argument #3 must support iteration